In [6]:
library("edgeR")
library("DESeq2")
library("dplyr")
library("ggplot2")
library("pheatmap")
library("RColorBrewer")
library("EnhancedVolcano")
library("glmpca")
library("PoiClaClu")
library("vidger")
library("apeglm")
library("HDF5Array")


Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from 'package:limma':

    plotMA


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

In [2]:
# BiocManager::install('EnhancedVolcano')

In [1]:
library("BiocParallel")
register(SnowParam(4))

In [12]:
g2t <- read.csv("./Transcript raw files/V6 (new annotation)/gx2tx_V6.txt", sep = " ", header = TRUE, col.names = c("geneID", "transcriptID"))
head(g2t)


,geneID,transcriptID
,<chr>,<chr>
1,AMEX60DD012306,AMEX60DD201012306.1
2,AMEX60DD012307,AMEX60DD201012307.1
3,AMEX60DD012308,AMEX60DD201012308.1
4,AMEX60DD012309,AMEX60DD201012309.1
5,AMEX60DD012310,AMEX60DD201012310.1
6,AMEX60DD012311,AMEX60DD301012311.1


In [13]:
t2symbol <- read.csv("./Transcript raw files/V6 (new annotation)/tx2symbol_V6.txt", sep = " ", header = FALSE, col.names = c("transcriptID", "symbol"))
# head(t2symbol)
filter(t2symbol, symbol == c('WNT7B'))

transcriptID,symbol
<chr>,<chr>
AMEX60DD201006490.2,WNT7B
AMEX60DD201006490.1,WNT7B
AMEX60DD201006490.3,WNT7B


In [14]:
repeat_content <- read.csv("./Transcript raw files/V6 (new annotation)/AmexG_v6_DD.repeat_content.txt", sep = " ", header = TRUE)

dim(repeat_content)
filter(repeat_content, gene_id == "AMEX60DD037919")

[1] 182117      7

gene_id,transcript_id,chromosome,start,end,strand,percent_repeat
<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>
AMEX60DD037919,AMEX60DD301037919.1,chr5p,686935374,687466532,+,0.02146
AMEX60DD037919,AMEX60DD201037919.2,chr5p,686939838,687466532,+,0.02164


In [15]:
# Add SYMBOL
repeat_content = merge(t2symbol,repeat_content, by.x="transcriptID", by.y="transcript_id")
dim(repeat_content)
head(repeat_content)

[1] 182117      8

,transcriptID,symbol,gene_id,chromosome,start,end,strand,percent_repeat
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>
1,AMEX60DD001003449.1,AMEX60DD003449,AMEX60DD003449,chr11p,39607084,39607461,-,0.00000
2,AMEX60DD001005685.1,AMEX60DD005685,AMEX60DD005685,chr11q,891290936,891291316,+,0.00000
3,AMEX60DD001009250.1,AMEX60DD009250,AMEX60DD009250,chr13p,239779496,239781253,+,0.00000
4,AMEX60DD001022557.1,AMEX60DD022557,AMEX60DD022557,chr2p,1208785144,1208785411,+,2.99625
5,AMEX60DD001025407.41,HNRNPUL2,AMEX60DD025407,chr2q,1083253575,1083255588,-,0.00000
6,AMEX60DD001026849.1,AMEX60DD026849,AMEX60DD026849,chr3p,163186719,163186985,-,100.00000


In [17]:
dim(filter(repeat_content, percent_repeat<=5))
dim(filter(repeat_content, percent_repeat<=10))

filter(repeat_content, symbol == "WNT7B")


[1] 127620      8

[1] 133940      8

transcriptID,symbol,gene_id,chromosome,start,end,strand,percent_repeat
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>
AMEX60DD201006490.1,WNT7B,AMEX60DD006490,chr12p,132702651,133282449,-,0.34978
AMEX60DD201006490.2,WNT7B,AMEX60DD006490,chr12p,132702651,133108508,-,0.61081
AMEX60DD201006490.3,WNT7B,AMEX60DD006490,chr12p,132709704,133108508,-,0.01479


In [98]:
## Filter and keep two columns

filtered_repeat_content = unique(filter(repeat_content, percent_repeat<=5)[c("symbol","gene_id")])
filter(filtered_repeat_content, symbol == "CHK1")

symbol,gene_id
<chr>,<chr>
CHK1,AMEX60DD000807
CHK1,AMEX60DD003682
CHK1,AMEX60DD004517
CHK1,AMEX60DD005669
CHK1,AMEX60DD005723
CHK1,AMEX60DD010457
CHK1,AMEX60DD018300
CHK1,AMEX60DD021029
CHK1,AMEX60DD021579


In [99]:
# rownames(filtered_repeat_content) = c(make.unique(filtered_repeat_content$symbol))
filtered_repeat_content$symbol = c(make.names(make.unique(filtered_repeat_content$symbol)))

In [101]:
write.csv(filtered_repeat_content,"./geneid_symbol_filtered_unique.csv",row.names=FALSE)

In [100]:
filter(filtered_repeat_content, symbol == "CHK1")

symbol,gene_id
<chr>,<chr>
CHK1,AMEX60DD000807
